# SQLALchemy 1.4 / 2.0 Tutorial
- [Working with Database Metadtaa](https://docs.sqlalchemy.org/en/14/tutorial/metadata.html)

In [1]:
from sqlalchemy import MetaData
metadata_obj = MetaData()

In [2]:
from sqlalchemy import Table, Column, Integer, String
user_table = Table(
    "user_account",
    metadata_obj,
    Column('id', Integer, primary_key=True),
    Column('name', String(30)),
    Column('fullname', String)
)

In [3]:
user_table

Table('user_account', MetaData(), Column('id', Integer(), table=<user_account>, primary_key=True, nullable=False), Column('name', String(length=30), table=<user_account>), Column('fullname', String(), table=<user_account>), schema=None)

In [8]:
user_table.name

'user_account'

In [7]:
user_table.c

In [11]:
user_table.columns

In [6]:
user_table.c.name

Column('name', String(length=30), table=<user_account>)

In [10]:
user_table.columns.name

Column('name', String(length=30), table=<user_account>)

In [12]:
user_table.primary_key

PrimaryKeyConstraint(Column('id', Integer(), table=<user_account>, primary_key=True, nullable=False))

In [13]:
from sqlalchemy import ForeignKey
address_table = Table(
    "address",
    metadata_obj,
    Column('id', Integer, primary_key=True),
    Column('user_id', ForeignKey('user_account.id'), nullable=False),
    Column('email_address', String, nullable=False)
)

In [15]:
from sqlalchemy import create_engine
engine = create_engine("sqlite+pysqlite:///:memory:", echo=True, future=True)
metadata_obj.create_all(engine)

2022-02-25 15:41:36,626 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-02-25 15:41:36,632 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2022-02-25 15:41:36,641 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-25 15:41:36,657 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2022-02-25 15:41:36,660 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-25 15:41:36,665 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2022-02-25 15:41:36,668 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-25 15:41:36,669 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("address")
2022-02-25 15:41:36,672 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-25 15:41:36,679 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	name VARCHAR(30), 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2022-02-25 15:41:36,681 INFO sqlalchemy.engine.Engine [no key 0.00209s] ()
2022-02-25 15:41:36,685 INFO sqlalchemy.engine.Engine 
C

In [16]:
from sqlalchemy.orm import registry
mapper_registry = registry()

In [17]:
mapper_registry.metadata

MetaData()

In [18]:
Base = mapper_registry.generate_base()

In [19]:
from sqlalchemy.orm import declarative_base
Base = declarative_base()

In [20]:
from sqlalchemy.orm import relationship
class User(Base):
    __tablename__ = 'user_account'
    id = Column(Integer, primary_key=True)
    name = Column(String(30))
    fullname = Column(String)
    addresses = relationship("Address", back_populates="user")
    def __repr__(self):
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"

class Address(Base):
    __tablename__ = 'address'
    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey('user_account.id'))
    user = relationship("User", back_populates="addresses")
    def __repr__(self):
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"

In [26]:
User.__table__

Table('user_account', MetaData(), Column('id', Integer(), table=<user_account>, primary_key=True, nullable=False), Column('name', String(length=30), table=<user_account>), Column('fullname', String(), table=<user_account>), schema=None)


In [22]:
Address.__table__

Table('address', MetaData(), Column('id', Integer(), table=<address>, primary_key=True, nullable=False), Column('email_address', String(), table=<address>, nullable=False), Column('user_id', Integer(), ForeignKey('user_account.id'), table=<address>), schema=None)

In [27]:
sandy = User(name='sandy', fullname='Sandy Cheeks')

In [28]:
sandy

User(id=None, name='sandy', fullname='Sandy Cheeks')

In [29]:
# emit CREATE statements given ORM registry
mapper_registry.metadata.create_all(engine)

# the identical MetaData object is also present on the
# declarative base
Base.metadata.create_all(engine)

2022-02-27 00:30:45,486 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-02-27 00:30:45,526 INFO sqlalchemy.engine.Engine COMMIT
2022-02-27 00:30:45,528 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-02-27 00:30:45,532 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2022-02-27 00:30:45,543 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-27 00:30:45,561 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2022-02-27 00:30:45,565 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-27 00:30:45,587 INFO sqlalchemy.engine.Engine COMMIT


In [31]:
class User(Base):
    __table__ = user_table

    addresses = relationship("Address", back_populates="user")

    def __repr__(self):
        return f"User({self.name!r}, {self.fullname!r})"

class Address(Base):
    __table__ = address_table

    user = relationship("User", back_populates="addresses")

    def __repr__(self):
        return f"Address({self.email_address!r})"

/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.User, and will be replaced in the string-lookup table.
  """Entry point for launching an IPython kernel.
/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.Address, and will be replaced in the string-lookup table.
  if __name__ == '__main__':


In [32]:
some_table = Table("some_table", metadata_obj, autoload_with=engine)

2022-02-27 00:36:59,813 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-02-27 00:36:59,821 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("some_table")
2022-02-27 00:36:59,827 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-27 00:36:59,835 INFO sqlalchemy.engine.Engine PRAGMA temp.table_xinfo("some_table")
2022-02-27 00:36:59,837 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-27 00:36:59,841 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("some_table")
2022-02-27 00:36:59,842 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-27 00:36:59,844 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("some_table")
2022-02-27 00:36:59,850 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-27 00:36:59,853 INFO sqlalchemy.engine.Engine ROLLBACK


NoSuchTableError: some_table